In [ ]:
%load_ext autoreload

In [ ]:
import os as _os
_os.chdir(_os.environ['PROJECT_ROOT'])
_os.path.realpath(_os.path.curdir)

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
from lib.pandas_util import idxwhere, align_indexes, invert_mapping
import matplotlib as mpl
import lib.plot
import statsmodels as sm
from statsmodels.stats.multitest import fdrcorrection
from tqdm import tqdm
import subprocess
from tempfile import mkstemp
import time
import subprocess
from itertools import chain
import os

In [ ]:
import sfacts as sf

In [ ]:
import lib.thisproject.data

In [ ]:
sns.set_context('talk')
plt.rcParams['figure.dpi'] = 50

In [ ]:
# Default file path forming for interactive use.

species = '102506'
centroid = 75

path = dict(
    gene_annotations=f"ref/midasdb_uhgg_gene_annotations/sp-{species}.gene{centroid}_annotations.tsv",
    reference_copy_number=f"ref/midasdb_uhgg_pangenomes/{species}/midas_gene{centroid}.reference_copy_number.nc",
    cluster_info=f"ref/midasdb_uhgg/pangenomes/{species}/cluster_info.txt",
    species_taxonomy="ref/gtpro/species_taxonomy_ext.tsv",
    midasdb_genomes="ref/uhgg_genomes_all_4644.tsv",
    gtpro_reference_genotype=f"data/species/sp-{species}/gtpro_ref.mgtp.nc",
)

path_exists = {}
for p in path:
    path_exists[path[p]] = os.path.exists(path[p])

assert all(path_exists.values()), '\n'.join(["Missing files:"] + [p for p in path_exists if not path_exists[p]])

In [ ]:
path['reference_copy_number']

In [ ]:
path['gtpro_reference_genotype']

In [ ]:
species_taxonomy = lib.thisproject.data.load_species_taxonomy(path["species_taxonomy"])
species_taxonomy.loc[species]

In [ ]:
ref_geno = sf.Genotype(
    sf.data.Metagenotype.load(path["gtpro_reference_genotype"])
    .to_estimated_genotype()
    .to_series()
    .unstack()
    .rename(lambda s: 'UHGG' + s[len('GUT_GENOME'):])
    .stack()
    .to_xarray()
)
print(ref_geno.sizes)

In [ ]:
pd.DataFrame(
    ref_geno.data.astype(int).values.T,
    columns=ref_geno.strain,
    index=ref_geno.position,
).to_csv('sp-102506.gtpro_ref.geno.tsv', sep='\t')

In [ ]:
ref_copy_number = xr.load_dataarray(path["reference_copy_number"])
ref_hit = pd.DataFrame(
    ref_copy_number.T > 0,
    columns=ref_copy_number.genome_id,
    index=ref_copy_number.gene_id,
)

print(ref_copy_number.sizes)

In [ ]:
ref_hit.astype(int).to_csv('sp-102506.midas_gene75.tsv', sep='\t')